In [1]:
!nvidia-smi

Mon Jul 11 14:36:08 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#version merge datasets v1
!pip install transformers==3.5.1
!pip install torch==1.4.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 4.2 MB/s 
     |████████████████████████████████| 880 kB 75.3 MB/s 
     |████████████████████████████████| 1.1 MB 66.8 MB/s 
     |████████████████████████████████| 2.9 MB 72.4 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=6f39bc4a433dc26a0e3d0107eb3db0796fd420dce63616614251bb03594e9b1e
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 753.4 MB 6.7 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0+cu113
    Uninstalling torch-1.11.0+cu113:
      Successfully uninstalled torch-1.11.0+cu113
ERROR: pip's dependency resolver does not curr

In [4]:
# from transformers import BertTokenizer, BertForPreTraining, BertForQuestionAnswering, BertModel, BertConfig
from transformers import XLMRobertaForQuestionAnswering, XLMRobertaTokenizer, XLMRobertaConfig
import torch
import torch.nn as nn
from transformers.data.metrics.squad_metrics import compute_predictions_log_probs, compute_predictions_logits, squad_evaluate
from transformers.data.processors.squad import SquadResult, SquadV1Processor, SquadV2Processor

In [5]:
model = XLMRobertaForQuestionAnswering.from_pretrained('/content/drive/MyDrive/Colab Notebooks/BLANC for Language Models/model/finetuned model/xlmr-viquad-finetuned-July11')
tokenizer = XLMRobertaTokenizer.from_pretrained('/content/drive/MyDrive/Colab Notebooks/BLANC for Language Models/model/finetuned model/xlmr-viquad-finetuned-July11')

In [ ]:
model.config

XLMRobertaConfig {
  "_name_or_path": "/content/drive/MyDrive/Colab Notebooks/BLANC for Language Models/model/pretrained model/xlmr-xquad-pretrained",
  "architectures": [
    "XLMRobertaForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.17.0.dev0",
  "type_vocab_size": 1,
  "vocab_size": 250002
}

In [6]:
processor = SquadV1Processor()

In [7]:
from transformers.data.processors.squad import squad_convert_examples_to_features

In [8]:
test_examples = processor.get_dev_examples('/content/drive/MyDrive/Colab Notebooks/MRC - VLSP/Dataset/ViQuADv1.1', 'test_ViQuAD.json')
test_features, test_dataset = squad_convert_examples_to_features(test_examples, 
                                                       tokenizer, 
                                                       max_seq_length = 384, 
                                                       doc_stride = 128,
                                                       max_query_length = 64,
                                                       is_training = False,
                                                       return_dataset = 'pt',
                                                       threads = 10
                                                       )

convert squad examples to features:   0%|          | 0/2210 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:1423: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:1423: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:1423: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:1423: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  Fut

In [9]:
import string
import re
import collections

from torch.utils.data import DataLoader, SequentialSampler
from tqdm import trange, tqdm
import os

In [10]:
device = torch.device('cuda')

model.to(device)

XLMRobertaForQuestionAnswering(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(250002, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
          

In [11]:
def to_list(tensor):
    return tensor.detach().cpu().tolist()
    

def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""

    def remove_articles(text):
        regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
        return re.sub(regex, " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))


def get_tokens(s):
    if not s:
        return []
    return normalize_answer(s).split()


def compute_exact(a_gold, a_pred):
    return int(normalize_answer(a_gold) == normalize_answer(a_pred))


def compute_f1(a_gold, a_pred):
    gold_toks = get_tokens(a_gold)
    pred_toks = get_tokens(a_pred)
    common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
    num_same = sum(common.values())
    if len(gold_toks) == 0 or len(pred_toks) == 0:
        # If either is no-answer, then F1 is 1 if they agree, 0 otherwise
        return int(gold_toks == pred_toks)
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(pred_toks)
    recall = 1.0 * num_same / len(gold_toks)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1

In [12]:
def evaluate(model, tokenizer, dev_dataset, dev_examples, dev_features):
    eval_sampler = SequentialSampler(dev_dataset)
    eval_dataloader = DataLoader(dev_dataset, sampler=eval_sampler, batch_size=12)
    all_results = []
#     start_time = timeit.default_timer()
    for batch in tqdm(eval_dataloader, desc="Evaluating"):
        model.eval()
        batch = tuple(t.to(device) for t in batch)
        with torch.no_grad():
            inputs = {
                "input_ids": batch[0],
                "attention_mask": batch[1],
                "token_type_ids": batch[2],
            }
            example_indices = batch[3]
            outputs = model(**inputs)
        for i, example_index in enumerate(example_indices):
            eval_feature = dev_features[example_index.item()]
            unique_id = int(eval_feature.unique_id)
#             for output in outputs:
#                 print(output)
            output = [to_list(output[i]) for output in outputs]
#             output = [to_list(output) for output in outputs]
            if len(output) >= 5:
                start_logits = output[0]
                start_top_index = output[1]
                end_logits = output[2]
                end_top_index = output[3]
                cls_logits = output[4]

                result = SquadResult(
                    unique_id,
                    start_logits,
                    end_logits,
                    start_top_index=start_top_index,
                    end_top_index=end_top_index,
                    cls_logits=cls_logits,
                )
            else:
                start_logits, end_logits = output
                result = SquadResult(unique_id, start_logits, end_logits)
            all_results.append(result)
    
    output_prediction_file = os.path.join("./", "predictions_{}.json".format(""))
    output_nbest_file = os.path.join("./", "nbest_predictions_{}.json".format(""))
    output_null_log_odds_file = os.path.join("./", "null_odds_{}.json".format(""))
    predictions = compute_predictions_logits(
            dev_examples,
            dev_features,
            all_results,
            20,
            300,
            False,
            output_prediction_file,
            output_nbest_file,
            output_null_log_odds_file,
            True,
            False,
            0.0,
            tokenizer,
        )
    # results = squad_evaluate(dev_examples, predictions)
    return dev_examples, predictions

In [13]:
test_examples, predictions = evaluate(model, tokenizer, test_dataset, test_examples, test_features)

Evaluating: 100%|██████████| 201/201 [03:06<00:00,  1.08it/s]


In [22]:
ls=[]
for example in test_examples:
    qas_id = example.qas_id
    gold_answers = [answer["text"] for answer in example.answers if normalize_answer(answer["text"])]

    if not gold_answers:
        # For unanswerable questions, only correct answer is empty string
        gold_answers = [""]

    if qas_id not in predictions:
        print("Missing prediction for %s" % qas_id)
        continue

    prediction = predictions[qas_id]
    # if prediction not in gold_answers:
    #     print('No.'+str(False_example_counter))
    #     False_example_counter = False_example_counter +1
    #     print('Question: ' + example.question_text)
    #     print('Gold Answer(s): ' + str(example.answers))
    #     print('Predicted Answer: '+ prediction)
    #     print('-------------------\n')
    f1_score = max(compute_f1(a, prediction) for a in gold_answers)
    if f1_score < 0.6:
        eg = {
          'Question': example.question_text,
          'Gold Answers': gold_answers,
          'Predicted Answer': prediction,
          'F1-score': f1_score
        }
        ls.append(eg)

from operator import itemgetter
ls.sort(key=itemgetter('F1-score'))

for idx, example in enumerate(ls):
    print('No.' + str(idx))
    print('Question: ' + example['Question'])
    print('Gold Answer(s): ' + str(example['Gold Answers']))
    print('Predicted Answer: '+ example['Predicted Answer'])
    print('F1 scores: ' + str(example['F1-score']))
    print('-------------------\n')

No.0
Question: Trước khi thuật ngữ Deutschland được sử dụng thì trước đó thuật ngữ nào được dùng?
Gold Answer(s): ['diutisciu land', 'diutisciu land', 'diutisciu land', 'diutisciu land']
Predicted Answer: tiếng Latinh
F1 scores: 0
-------------------

No.1
Question: Phía nam của Đan Mạch giáp với quốc gia nào?
Gold Answer(s): ['Đức', 'Áo', 'Đức', 'Đức']
Predicted Answer: Thụy Sĩ
F1 scores: 0
-------------------

No.2
Question: Chức vụ thủ tướng tại Đức là do tổ chức nào chọn ra?
Gold Answer(s): ['Bundespräsident', 'Bundespräsident', 'Bundespräsident', 'Bundespräsident']
Predicted Answer: Bundestag
F1 scores: 0
-------------------

No.3
Question: Nguồn gốc của hơn một nửa năng lượng tại Đức là gì??
Gold Answer(s): ['nhập khẩu', 'nhập khẩu', 'năng lượng sơ cấp được nhập khẩu', 'nhập khẩu']
Predicted Answer: dầu
F1 scores: 0
-------------------

No.4
Question: Trong thập niên 1970, trong loạt các phim Đức thành công thì nổi bật nhất là bộ phim có tên tiếng việt là gì?
Gold Answer(s): ['Ph

In [20]:
F1_range_v1 =['0.0-0.2','0.2-0.4','0.4-0.6','0.6-0.8','0.8-0.9','0.9-1.0', 'T']
n_ans=[0,0,0,0,0,0,0]
gold_answer_average_len = [0,0,0,0,0,0,0]
predicted_answer_average_len = [0,0,0,0,0,0,0]

n_low_0_9 = 0
n_high_0_9 = 0

for example in test_examples:
    qas_id = example.qas_id
    gold_answers = [answer["text"] for answer in example.answers if normalize_answer(answer["text"])]

    if not gold_answers:
        # For unanswerable questions, only correct answer is empty string
        gold_answers = [""]

    if qas_id not in predictions:
        print("Missing prediction for %s" % qas_id)
        continue

    prediction = predictions[qas_id]

    
    predicted_answer_len = len(normalize_answer(prediction).split())
    gold_answer_len = min(len(normalize_answer(a).split()) for a in gold_answers)

    f1_scores = max(compute_f1(a, prediction) for a in gold_answers)
    if f1_scores < 0.9:
        n_low_0_9 = n_low_0_9 + 1
    else:
        n_high_0_9 = n_high_0_9 + 1

    if f1_scores < 0.2:
        n_ans[0] = n_ans[0] + 1
        gold_answer_average_len[0] = gold_answer_average_len[0] + gold_answer_len
        predicted_answer_average_len[0] = predicted_answer_average_len[0] + predicted_answer_len
    elif f1_scores < 0.4:
        n_ans[1] = n_ans[1] + 1
        gold_answer_average_len[1] = gold_answer_average_len[1] + gold_answer_len
        predicted_answer_average_len[1] = predicted_answer_average_len[1] + predicted_answer_len
    elif f1_scores < 0.6:
        n_ans[2] = n_ans[2] + 1
        gold_answer_average_len[2] = gold_answer_average_len[2] + gold_answer_len
        predicted_answer_average_len[2] = predicted_answer_average_len[2] + predicted_answer_len
    elif f1_scores < 0.8:
        n_ans[3] = n_ans[3] + 1
        gold_answer_average_len[3] = gold_answer_average_len[3] + gold_answer_len
        predicted_answer_average_len[3] = predicted_answer_average_len[3] + predicted_answer_len
    elif f1_scores < 0.9:
        n_ans[4] = n_ans[4] + 1
        gold_answer_average_len[4] = gold_answer_average_len[4] + gold_answer_len
        predicted_answer_average_len[4] = predicted_answer_average_len[4] + predicted_answer_len
    elif f1_scores < 1:
        n_ans[5] = n_ans[5] + 1
        gold_answer_average_len[5] = gold_answer_average_len[5] + gold_answer_len
        predicted_answer_average_len[5] = predicted_answer_average_len[5] + predicted_answer_len
    else:
        n_ans[6] = n_ans[6] + 1
        gold_answer_average_len[6] = gold_answer_average_len[6] + gold_answer_len
        predicted_answer_average_len[6] = predicted_answer_average_len[6] + predicted_answer_len

gold_answer_average_len = [i / j for i, j in zip(gold_answer_average_len, n_ans)]
predicted_answer_average_len = [i / j for i, j in zip(predicted_answer_average_len, n_ans)]

print('# answers F1-score < 0.9: ' + str(n_low_0_9))
print('# answers F1-score >= 0.9: ' + str(n_high_0_9))
print('--------------\n')
for i in range(len(F1_range_v1)):
    print('F1-score span: ' + F1_range_v1[i])
    print('# examples: ' + str(n_ans[i]))
    print('gold_answer_average_len: ' + str(gold_answer_average_len[i]))
    print('predicted_answer_average_len: ' + str(predicted_answer_average_len[i]))
    print('---------------\n')



# answers F1-score < 0.9: 556
# answers F1-score >= 0.9: 1654
--------------

F1-score span: 0.0-0.2
# examples: 116
gold_answer_average_len: 9.35344827586207
predicted_answer_average_len: 20.198275862068964
---------------

F1-score span: 0.2-0.4
# examples: 65
gold_answer_average_len: 11.415384615384616
predicted_answer_average_len: 19.96923076923077
---------------

F1-score span: 0.4-0.6
# examples: 112
gold_answer_average_len: 7.330357142857143
predicted_answer_average_len: 15.473214285714286
---------------

F1-score span: 0.6-0.8
# examples: 142
gold_answer_average_len: 8.816901408450704
predicted_answer_average_len: 12.401408450704226
---------------

F1-score span: 0.8-0.9
# examples: 121
gold_answer_average_len: 8.694214876033058
predicted_answer_average_len: 12.239669421487603
---------------

F1-score span: 0.9-1.0
# examples: 133
gold_answer_average_len: 14.977443609022556
predicted_answer_average_len: 20.278195488721803
---------------

F1-score span: T
# examples: 1521
g

## Nguyên nhân chính dẫn đến sự sai khác là do trung bình 1 answer predict bị sai sẽ có độ dài khoảng 15-20 ký tự (tương tự độ dài answer của XQuAD), trong khi đó độ dài trung bình answer của viquad là 8.2